In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from colorama import Fore, Style
# Set GPU device
# Configure TensorFlow to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

# Print GPU devices
tf.test.gpu_device_name()
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    print(gpu)

# Print session data
print(sess)

def print_decorative_log(message, color=Fore.BLUE, style=Style.RESET_ALL):
    line_length = len(message) + 4  # Length of the message plus padding on both sides
    decorative_line = "#" * line_length
    print(color + decorative_line)
    print(f"# {message} #")
    print(decorative_line + style)


# Load dataset
print_decorative_log("Loading Dataset", Fore.YELLOW)
df = pd.read_csv('merged_dataset.csv')
# Define the column names
columns = ['mfcc_' + str(i) for i in range(1, 301)] + ['label']

# Assign the column names to the DataFrame
df.columns = columns


Num GPUs Available:  0
###################
# Loading Dataset #
###################


In [13]:

print_decorative_log("Dataset Loaded", Fore.GREEN)
from sklearn.preprocessing import StandardScaler
# Configure TensorFlow to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

# # Dropping rows with label equal to 3
# df = df[df['label'] != 3]

print_decorative_log("Dataset preparation and splitting", Fore.YELLOW)
X = df.drop('label', axis=1).values.astype(np.float32)  # Features
#y = df['label'].values  # Labels
y = df['label'].values.astype(np.float32)  # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a standard scaler
print_decorative_log("Normalization & Standardization", Fore.YELLOW)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# # Apply SMOTE oversampling to address class imbalance
# print_decorative_log("Applying SMOTE", Fore.YELLOW)
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
#Define ADASYN oversampling
print_decorative_log("ADASYN Oversampling", Fore.YELLOW)
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_scaled, y_train)

# Create GPU training
strategy = tf.distribute.OneDeviceStrategy('GPU:0')  # Use the first GPU


##################
# Dataset Loaded #
##################
#####################################
# Dataset preparation and splitting #
#####################################
###################################
# Normalization & Standardization #
###################################
#######################
# ADASYN Oversampling #
#######################


In [15]:
print(X_train.shape)
print(X_train_resampled.shape)

(339825, 300)
(1335481, 300)


In [27]:
# Reshape the input data to have an additional dimension
X_train_resampled = np.reshape(X_train_resampled, (X_train_resampled.shape[0], X_train_resampled.shape[1], 1))
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
# RNN (LSTM) model
print_decorative_log("RNN Model Training", Fore.YELLOW)
with strategy.scope():
    rnn_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(X_train_resampled.shape[1],1)),
        tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])

    rnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    rnn_model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    
    



######################
# RNN Model Training #
######################
Epoch 1/10
33387/33387 [==============================] - 3916s 117ms/step - loss: 0.4924 - accuracy: 0.8159 - val_loss: 2.3420 - val_accuracy: 0.3597
Epoch 2/10
33387/33387 [==============================] - 4122s 123ms/step - loss: 0.1417 - accuracy: 0.9553 - val_loss: 2.2152 - val_accuracy: 0.4851
Epoch 3/10
33387/33387 [==============================] - 3928s 118ms/step - loss: 0.5712 - accuracy: 0.7648 - val_loss: 2.7345 - val_accuracy: 7.5628e-04
Epoch 4/10
33387/33387 [==============================] - 3910s 117ms/step - loss: 1.1221 - accuracy: 0.4998 - val_loss: 2.5596 - val_accuracy: 0.0000e+00
Epoch 5/10
33387/33387 [==============================] - 3903s 117ms/step - loss: 0.5330 - accuracy: 0.8033 - val_loss: 2.0495 - val_accuracy: 0.2338
Epoch 6/10
33387/33387 [==============================] - 3891s 117ms/step - loss: 0.2145 - accuracy: 0.9293 - val_loss: 2.1012 - val_accuracy: 0.3566
Epoch 7/10
33387/

In [30]:
#Evaluate the model on the test data
print_decorative_log("RNN Model Evaluation", Fore.YELLOW)
rnn_predictions = rnn_model.predict(X_test_scaled)

# Get the predicted class labels
predicted_labels = np.argmax(rnn_predictions, axis=1)

# Evaluate the predictions
rnn_report = classification_report(y_test, predicted_labels)
print("RNN Classification Report:")
print(rnn_report)

########################
# RNN Model Evaluation #
########################
2655/2655 [==============================] - 104s 39ms/step
RNN Classification Report:
              precision    recall  f1-score   support

         0.0       0.01      0.39      0.02       512
         1.0       0.02      0.58      0.04       540
         2.0       0.01      0.02      0.02       506
         3.0       0.99      0.60      0.75     83399

    accuracy                           0.60     84957
   macro avg       0.26      0.40      0.21     84957
weighted avg       0.98      0.60      0.74     84957



In [29]:
# Save the model as an HDF5 file
rnn_model.save("rnn_epilepsy_prediction_model.h5")
print_decorative_log("RNN Model Exported", Fore.GREEN)

######################
# RNN Model Exported #
######################


c:\Users\MUSA\anaconda3\envs\NeuroGuard\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
# Reshape the input data to have an additional dimension
X_train_resampled = np.reshape(X_train_resampled, (X_train_resampled.shape[0], X_train_resampled.shape[1], 1))
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
# CNN model
print_decorative_log("CNN Model Training", Fore.YELLOW)
with strategy.scope():
    cnn_model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_resampled.shape[1], 1)),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])

    cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    cnn_model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    

######################
# CNN Model Training #
######################
Epoch 1/10
33387/33387 [==============================] - 319s 9ms/step - loss: 0.0708 - accuracy: 0.9780 - val_loss: 1.2430 - val_accuracy: 0.7631
Epoch 2/10
33387/33387 [==============================] - 307s 9ms/step - loss: 0.0165 - accuracy: 0.9957 - val_loss: 1.1361 - val_accuracy: 0.7794
Epoch 3/10
33387/33387 [==============================] - 311s 9ms/step - loss: 0.0108 - accuracy: 0.9972 - val_loss: 0.8219 - val_accuracy: 0.8387
Epoch 4/10
33387/33387 [==============================] - 318s 10ms/step - loss: 0.0086 - accuracy: 0.9978 - val_loss: 0.6053 - val_accuracy: 0.8670
Epoch 5/10
33387/33387 [==============================] - 318s 10ms/step - loss: 0.0068 - accuracy: 0.9982 - val_loss: 0.6690 - val_accuracy: 0.8586
Epoch 6/10
33387/33387 [==============================] - 312s 9ms/step - loss: 0.0058 - accuracy: 0.9985 - val_loss: 0.2925 - val_accuracy: 0.9283
Epoch 7/10
33387/33387 [=================

In [20]:
# Evaluate the model on the test data
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
print_decorative_log("CNN Model Evaluation", Fore.YELLOW)
cnn_predictions = cnn_model.predict(X_test_scaled)
# Get the predicted class labels
predicted_labels = np.argmax(cnn_predictions, axis=1)

cnn_report = classification_report(y_test, predicted_labels)
print("CNN Classification Report:")
print(cnn_report)

########################
# CNN Model Evaluation #
########################
2655/2655 [==============================] - 8s 3ms/step
CNN Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.88      0.84       512
         1.0       0.86      0.92      0.89       540
         2.0       0.78      0.85      0.81       506
         3.0       1.00      1.00      1.00     83399

    accuracy                           1.00     84957
   macro avg       0.86      0.91      0.89     84957
weighted avg       1.00      1.00      1.00     84957



In [21]:
import pickle
# Export the trained SVM model to a pickle file
with open('cnn_epilepsy_prediction_model.pkl', 'wb') as file:
    pickle.dump(cnn_model, file)
print_decorative_log("CNN Model Exported", Fore.GREEN) 

######################
# CNN Model Exported #
######################


In [26]:
import keras

print("Keras version:", keras.__version__)

Keras version: 2.10.0


In [22]:
# Save the model as an HDF5 file
cnn_model.save("cnn_epilepsy_prediction_model.h5")
print_decorative_log("CNN Model Exported", Fore.GREEN)

######################
# CNN Model Exported #
######################


c:\Users\MUSA\anaconda3\envs\NeuroGuard\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
len(np.unique(y))


(84957,)